In [1]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd
import tifffile as tiff

2023-07-28 09:34:39.936208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


**You will need `fastplotlib` installed for the visualizations**

In [2]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [3]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

**Even on windows just use `/`, you do not have to worry about the annoying issue of `\\` and `\` on windows if you use `pathlib` :D**

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/Users/mirckuz/caiman_data/")

PosixPath('/Users/mirckuz/caiman_data')

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [9]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [10]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


# Path to the input movie

An input movie must be anywhere within `raw data path` or `batch path`. We will use the Sue 2p example.

In [ ]:
tif = tiff.imread('/Users/mirckuz/caiman_data/example_movies/hz09_drift_corrected_movie_0000.tif')
print(tif.shape)
single_plane_movie = tif[:,30,:,:]
print(single_plane_movie.shape)
tiff.imsave('/Users/mirckuz/caiman_data/example_movies/hz09_reg_singPlane.tif', single_plane_movie)

In [11]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/hz09_reg_singPlane.tif")

# CNMF

## Perform CNMF using the good mcorr output.

First, the params for CNMF. Put the CNMF params within the `main` key, `refit` is if you want to run CNMF for a second iteration.

In [12]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 1, # framerate, very important!
            'p': 2, #order of autoregressive system (2 if peak is after some frames from stimulus)
            'nb': 2, #global background components
            'merge_thr': 0.85, #max correlation allowed
            'rf': 15, #half_size patch
            'stride': 5, # "stride" for cnmf, "strides" for mcorr - strides overlap
            'K': 30,
            'gSig': [3, 3], #expected half size of neuron in px
            'ssub': 2, #spatial subsampling during initialization
            'tsub': 2, #temporal subsampling during initialization
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

### We can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [13]:
from itertools import product

# variants of several parameters
gSig_variants = [2, 3]
K_variants = [30, 40, 50]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name='hz09_reg_singPlane',
        input_movie_path=movie_path,
        params=new_params_cnmf
    )

df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 30, 'gSig': (2, 2), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,ac634b88-f7a8-41e5-8a92-d7df84332b29
1,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 40, 'gSig': (2, 2), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,646a6808-df63-4889-93a7-6ca55cd3e13e
2,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 50, 'gSig': (2, 2), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,2b83e0c0-034c-409a-a506-fb1cf04b9b09
3,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 30, 'gSig': (3, 3), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,d5982b7d-e6df-4fb0-9518-de0d9d39df55
4,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 40, 'gSig': (3, 3), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,0842477f-8a88-46d9-8e43-050f9e1ccb01
5,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 50, 'gSig': (3, 3), 'ssub': 2, 't...",None,2023-07-28T09:37:13,None,None,None,5c71fd85-eae1-479e-a751-1e7e59ea3b1d


## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [14]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])

/var/folders/7y/k8g92w294ts1x3qs46crmd_40000gn/T/ipykernel_29520/3330105589.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])


0    {'K': 30, 'gSig': (2, 2)}
1    {'K': 40, 'gSig': (2, 2)}
2    {'K': 50, 'gSig': (2, 2)}
3    {'K': 30, 'gSig': (3, 3)}
4    {'K': 40, 'gSig': (3, 3)}
5    {'K': 50, 'gSig': (3, 3)}
Name: params, dtype: object

# Run the added `cnmf` batch items

### First, this is how you can filter a pandas DataFrame using multiple columns. This gives you the rows (batch items) using the "cnmf" `"algo"` and those that match a particular `"item_name"`

In [15]:
for i, row in df[
    (df["algo"] == "cnmf") &
    (df["item_name"] == df.iloc[0]["item_name"])
].iterrows():
    
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          hz09_reg_singPlane
input_movie_path                example_movies/hz09_reg_singPlane.tif
params              {'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr...
outputs                                                          None
added_time                                        2023-07-28T09:37:13
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             ac634b88-f7a8-41e5-8a92-d7df84332b29
Name: 0, dtype: object
With params:{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 30, 'gSig': (2, 2), 'ssub': 2, 'tsub': 2, 'method_init': 'greedy_roi', 'min_SNR': 2.0, '

/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '


performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /Users/mirckuz/caiman_data/model/cnn_model.json
188/188 [==============================] - 3s 15ms/step
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/ac634b88-f7a8-41e5-8a92-d7df84332b29_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/ac634b88-f7a8-41e5-8a92-d7df84332b29_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/ac634b88-f7a8-41e5-8a92-d7df84332b29_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/ac634b88-f7a8-41e5-8a92-d7df84332b29_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/ac634b88-f7a8-41e5-8a92-d7df84332b29_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200

/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '


performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /Users/mirckuz/caiman_data/model/cnn_model.json
255/255 [==============================] - 4s 15ms/step
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/646a6808-df63-4889-93a7-6ca55cd3e13e_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/646a6808-df63-4889-93a7-6ca55cd3e13e_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/646a6808-df63-4889-93a7-6ca55cd3e13e_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/646a6808-df63-4889-93a7-6ca55cd3e13e_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/646a6808-df63-4889-93a7-6ca55cd3e13e_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200

...

performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /Users/mirckuz/caiman_data/model/cnn_model.json
210/210 [==============================] - 3s 14ms/step
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/d5982b7d-e6df-4fb0-9518-de0d9d39df55_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/d5982b7d-e6df-4fb0-9518-de0d9d39df55_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/d5982b7d-e6df-4fb0-9518-de0d9d39df55_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/d5982b7d-e6df-4fb0-9518-de0d9d39df55_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/d5982b7d-e6df-4fb0-9518-de0d9d39df55_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200

/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '


performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /Users/mirckuz/caiman_data/model/cnn_model.json
285/285 [==============================] - 5s 19ms/step
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/0842477f-8a88-46d9-8e43-050f9e1ccb01_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/0842477f-8a88-46d9-8e43-050f9e1ccb01_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/0842477f-8a88-46d9-8e43-050f9e1ccb01_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/0842477f-8a88-46d9-8e43-050f9e1ccb01_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/0842477f-8a88-46d9-8e43-050f9e1ccb01_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200

/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/Users/mirckuz/mambaforge/envs/caiman/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '


performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /Users/mirckuz/caiman_data/model/cnn_model.json
359/359 [==============================] - 5s 15ms/step
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200.mmap
Decode mmap filename /Users/mirckuz/caiman_data/example_movies/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_cnmf-memmap_d1_296_d2_540_d3_1_order_C_frames_200

### We now have CNMF outputs

In [16]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmf"]

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 30, 'gSig': (2, 2), 'ssub': 2, 't...",{'mean-projection-path': ac634b88-f7a8-41e5-8a92-d7df84332b29/ac634b88-f7a8-41e5-8a92-d7df84332b29_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T09:45:46,484.94 sec,None,ac634b88-f7a8-41e5-8a92-d7df84332b29
1,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 40, 'gSig': (2, 2), 'ssub': 2, 't...",{'mean-projection-path': 646a6808-df63-4889-93a7-6ca55cd3e13e/646a6808-df63-4889-93a7-6ca55cd3e13e_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T09:57:49,707.4 sec,None,646a6808-df63-4889-93a7-6ca55cd3e13e
2,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 50, 'gSig': (2, 2), 'ssub': 2, 't...",{'mean-projection-path': 2b83e0c0-034c-409a-a506-fb1cf04b9b09/2b83e0c0-034c-409a-a506-fb1cf04b9b09_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T10:14:12,966.43 sec,None,2b83e0c0-034c-409a-a506-fb1cf04b9b09
3,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 30, 'gSig': (3, 3), 'ssub': 2, 't...",{'mean-projection-path': d5982b7d-e6df-4fb0-9518-de0d9d39df55/d5982b7d-e6df-4fb0-9518-de0d9d39df55_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T10:26:52,743.26 sec,None,d5982b7d-e6df-4fb0-9518-de0d9d39df55
4,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 40, 'gSig': (3, 3), 'ssub': 2, 't...",{'mean-projection-path': 0842477f-8a88-46d9-8e43-050f9e1ccb01/0842477f-8a88-46d9-8e43-050f9e1ccb01_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T10:44:37,1055.61 sec,None,0842477f-8a88-46d9-8e43-050f9e1ccb01
5,cnmf,hz09_reg_singPlane,example_movies/hz09_reg_singPlane.tif,"{'main': {'fr': 1, 'p': 2, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 5, 'K': 50, 'gSig': (3, 3), 'ssub': 2, 't...",{'mean-projection-path': 5c71fd85-eae1-479e-a751-1e7e59ea3b1d/5c71fd85-eae1-479e-a751-1e7e59ea3b1d_mean_projection.n...,2023-07-28T09:37:13,2023-07-28T11:09:52,1499.22 sec,None,5c71fd85-eae1-479e-a751-1e7e59ea3b1d


In [17]:
# see which batch items completed succcessfully
df[df["algo"] == "cnmf"]["outputs"].apply(lambda x: x["success"])

0    True
1    True
2    True
3    True
4    True
5    True
Name: outputs, dtype: bool